In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests, time, json, pytz
import pandas as pd
import SpaceTraders as ST
from SpaceTraders import F_utils, F_nav, F_trade, io
from datetime import datetime

In [3]:
ST.get_request('/my/agent').json()

{'data': {'accountId': 'cmhhh9gk1002vtm16u5rl9pf0',
  'symbol': 'RYVIOS',
  'headquarters': 'X1-QT34-A1',
  'credits': 175000,
  'startingFaction': 'COSMIC',
  'shipCount': 2}}

In [35]:
ST.get_request('/agents/HIVE').json()

{'data': {'symbol': 'HIVE',
  'headquarters': 'X1-QT34-A1',
  'credits': 7581405,
  'startingFaction': 'COSMIC',
  'shipCount': 17}}

In [4]:
survey = ST.survey_waypoint('RYVIOS-1')
survey

{'signature': 'X1-TP30-AB5E-37C2F5',
 'symbol': 'X1-TP30-AB5E',
 'deposits': [{'symbol': 'ALUMINUM_ORE'},
  {'symbol': 'SILICON_CRYSTALS'},
  {'symbol': 'SILICON_CRYSTALS'}],
 'expiration': '2025-11-03T11:16:04.368Z',
 'size': 'SMALL'}

In [10]:
mining_r.json()

{'data': {'extraction': {'shipSymbol': 'RYVIOS-1',
   'yield': {'symbol': 'COPPER_ORE', 'units': 4}},
  'cooldown': {'shipSymbol': 'RYVIOS-1',
   'totalSeconds': 80,
   'remainingSeconds': 79,
   'expiration': '2025-11-03T09:53:13.070Z'},
  'cargo': {'capacity': 40,
   'units': 9,
   'inventory': [{'symbol': 'QUARTZ_SAND',
     'name': 'Quartz Sand',
     'description': 'High-purity quartz sand used in the production of glass and ceramics.',
     'units': 5},
    {'symbol': 'COPPER_ORE',
     'name': 'Copper Ore',
     'description': 'A valuable ore used in the production of copper and other alloys. Copper ore is an essential component in the production of electronic components and wiring.',
     'units': 4}]},
  'events': [],
  'modifiers': []}}

In [13]:
r = ST.post_request('/my/ships/RYVIOS-1/scan/waypoints')

In [14]:
r.json()

{'data': {'cooldown': {'shipSymbol': 'RYVIOS-1',
   'totalSeconds': 80,
   'remainingSeconds': 79,
   'expiration': '2025-11-10T19:20:09.625Z'},
  'waypoints': [{'symbol': 'X1-QT34-A1',
    'type': 'PLANET',
    'systemSymbol': 'X1-QT34',
    'x': 23,
    'y': -13,
    'orbitals': [{'symbol': 'X1-QT34-A2'},
     {'symbol': 'X1-QT34-A3'},
     {'symbol': 'X1-QT34-A4'}],
    'traits': [{'symbol': 'VOLCANIC',
      'name': 'Volcanic',
      'description': 'A volatile world marked by intense volcanic activity, creating a hazardous environment with the potential for valuable resource extraction, such as rare metals and geothermal energy.'},
     {'symbol': 'OUTPOST',
      'name': 'Outpost',
      'description': 'A small, remote settlement providing essential services and a safe haven for travelers passing through.'},
     {'symbol': 'VIBRANT_AURORAS',
      'name': 'Vibrant Auroras',
      'description': "A celestial light show caused by the interaction of charged particles with the waypoi

In [5]:
ST.get_contracts()

[ERROR] Failed to fetch contracts.
 [INFO] {'message': 'Route GET:/https://api.spacetraders.io/v2/my/contracts not found', 'error': 'Not Found', 'statusCode': 404}


[]

In [33]:
ST.deliver_cargo('cmhj5v575ir8eri738qzphmt3', 'RYVIOS-1', 'IRON_ORE')

[ERROR] Contract cmhj5v575ir8eri738qzphmt3 doesn't accept IRON_ORE. Aborting delivery.


False

In [5]:
F_nav.refuel_ship('RYVIOS-1')
F_nav.navigate_in_system('RYVIOS-1', 'X1-TP30-D41', 'BURN', verbose=True)

[INFO] Ship RYVIOS-1 refueled 0 units. Total cost: 0 cr (72 cr/u).
[ERROR] Ship RYVIOS-1 is already at destination X1-TP30-D41. Navigation cancelled.


False

In [9]:
F_nav.get_ship_nav('RYVIOS-1')

{'systemSymbol': 'X1-TP30',
 'waypointSymbol': 'X1-TP30-D41',
 'route': {'destination': {'symbol': 'X1-TP30-D41',
   'type': 'PLANET',
   'systemSymbol': 'X1-TP30',
   'x': 41,
   'y': -75},
  'origin': {'symbol': 'X1-TP30-H52',
   'type': 'MOON',
   'systemSymbol': 'X1-TP30',
   'x': 40,
   'y': -19},
  'departureTime': '2025-11-04T17:42:20.961Z',
  'arrival': '2025-11-04T17:42:55.961Z'},
 'status': 'DOCKED',
 'flightMode': 'BURN'}

In [7]:
import sqlite3

DB_PATH = './data/STDB.db'

In [8]:
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

In [9]:
wps = ST.get_waypoints_in_system('X1-TP30')

waypoint_data  = list()
traits_data    = list()
modifiers_data = list()

for wp in wps:
    waypoint_data.append({
      "symbol": wp["symbol"],
      "type": wp["type"],
      "systemSymbol": wp["systemSymbol"],
      "x": wp["x"],
      "y": wp["y"],
      "isUnderConstruction": wp["isUnderConstruction"]})
    traits_data.extend([{"waypointSymbol": wp["symbol"], **t} for t in wp["traits"]])
    modifiers_data.extend([{"waypointSymbol": wp["symbol"], **m} for m in wp["modifiers"]])

In [26]:
pd.DataFrame(traits_data).to_sql('TRAITS', conn)

220

In [20]:
pd.read_sql_query("SELECT symbol, x, y FROM WAYPOINTS LIMIT 10", conn)

,symbol,x,y
0,X1-TP30-A1,-19,-14
1,X1-TP30-AB5E,10,-23
2,X1-TP30-B6,-35,-185
3,X1-TP30-B7,-255,-229
4,X1-TP30-B8,-105,-358
5,X1-TP30-B9,-188,-300
6,X1-TP30-B10,-119,-338
7,X1-TP30-B11,-95,-362
8,X1-TP30-B12,213,-234
9,X1-TP30-B13,-37,-348


In [25]:
ship        = "RYVIOS-1"
source_mkt  = "X1-TP30-D42"
sink_mkt    = "X1-TP30-C39"
target_good = "SHIP_PLATING"

In [34]:
def print_nav_info(ship):
    nav = F_nav.get_ship_nav(ship)
    if nav['status'] == 'IN_TRANSIT':
        arrival_time = nav['route']['arrival'][:-1]
        delta_time = datetime.fromisoformat(arrival_time) - datetime.now()
        print(ship, "is navigating to", nav['route']['destination']['symbol'], "- ETA", nav['route']['arrival'], f"({delta_time.total_seconds()} seconds remaining).")
    else:
        print(f"{ship} is {nav['status']} at {nav['waypointSymbol']}.")
        inv = F_trade.get_ship_cargo(ship)['inventory']
        if len(inv):
            print("[CARGO]")
            for i in inv:
                print(f"  - {i['units']} {i['symbol']}")

In [28]:
sys = F_utils.system_from_wp(F_nav.get_ship_waypoint('RYVIOS-2'))
path = F_nav.get_waypoints_in_system(sys, traits=['MARKETPLACE'])

In [35]:
print_nav_info('RYVIOS-1')

RYVIOS-1 is DOCKED at X1-TP30-A1.


In [7]:
F_trade.purchase_cargo('RYVIOS-1', 'SHIP_PARTS', 15)

[INFO] RYVIOS-1 bought 6 SHIP_PARTS for 17220 cr.
[INFO] RYVIOS-1 bought 6 SHIP_PARTS for 17316 cr.
[INFO] RYVIOS-1 bought 3 SHIP_PARTS for 8718 cr.
[INFO] RYVIOS-1 finished selling 15 SHIP_PARTS.


True

In [36]:
F_nav.refuel_ship('RYVIOS-1')
F_nav.navigate_in_system('RYVIOS-1', 'X1-TP30-A2', 'BURN')

[INFO] Ship RYVIOS-1 refueled 2 units. Total cost: 144 cr (72 cr/u).
[INFO] Ship RYVIOS-1 en-route to X1-TP30-A2. Arrival at 2025-11-04T20:20:51.918 (15.0 seconds).


True

In [16]:
F_trade.sell_all_goods('RYVIOS-1', ['SHIP_PARTS'])

[INFO] Ship RYVIOS-1 sold 6 SHIP_PARTS @ 7976 for a total of 47856 credits.
[INFO] Ship RYVIOS-1 sold 6 SHIP_PARTS @ 7878 for a total of 47268 credits.
[INFO] Ship RYVIOS-1 sold 3 SHIP_PARTS @ 7757 for a total of 23271 credits.


In [13]:
goodsA = {'SHIP_PLATING': 15}
[(g, goodsA[g]) for g in goodsA.keys()]

[('SHIP_PLATING', 15)]

In [30]:
best_hop_query = \
"""
select
symbol, source, sink, tradeVolume
from TRADE_SYSTEM_MARGINS
--where source = "X1-TP30-A1"
where distance < 101
and margin > 1
and tradeVolume >= 10
and src_supply = "ABUNDANT" and sink_supply = "SCARCE"
order by margin desc
"""
r = ST.DB_CURSOR().execute(best_hop_query).fetchone()

In [31]:
script_cmd = f"scripts.trade_loop('RYVIOS-1', '{r[1]}', {{'{r[0]}': {r[3]}}}, '{r[2]}')"
print(script_cmd)

scripts.trade_loop('RYVIOS-1', 'X1-TP30-H54', {'JEWELRY': 20}, 'X1-TP30-A1')


In [32]:
F_nav.get_waypoints_in_system('X1-TP30', traits=['SHIPYARD'])

[{'symbol': 'X1-TP30-A2',
  'type': 'MOON',
  'systemSymbol': 'X1-TP30',
  'x': -19,
  'y': -14,
  'orbitals': [],
  'traits': [{'symbol': 'FROZEN',
    'name': 'Frozen',
    'description': 'An ice-covered world with frigid temperatures, providing unique research opportunities and resources such as ice water, ammonia ice, and other frozen compounds.'},
   {'symbol': 'SCATTERED_SETTLEMENTS',
    'name': 'Scattered Settlements',
    'description': 'A collection of dispersed communities, each independent yet connected through trade and communication networks.'},
   {'symbol': 'THIN_ATMOSPHERE',
    'name': 'Thin Atmosphere',
    'description': 'A location with a sparse atmosphere, making it difficult to support life without specialized life-support systems.'},
   {'symbol': 'TERRAFORMED',
    'name': 'Terraformed',
    'description': 'A waypoint that has been artificially transformed to support life, showcasing the engineering prowess of its inhabitants and providing a hospitable environm

In [53]:
# Dock first
ship = 'RYVIOS-1'
if F_nav.dock_ship(ship):
    cur_wp = F_nav.get_ship_waypoint(ship)
    sy_data = F_trade.get_shipyard_info(cur_wp)

In [54]:
sy_df = pd.DataFrame.from_dict(sy_data['ships'])
sy_df

,type,name,description,supply,purchasePrice,frame,reactor,engine,modules,mounts,crew,activity
0,SHIP_PROBE,Probe Satellite,"A small, unmanned spacecraft that can be launc...",HIGH,22288,"{'symbol': 'FRAME_PROBE', 'name': 'Probe', 'co...","{'symbol': 'REACTOR_SOLAR_I', 'name': 'Solar R...","{'symbol': 'ENGINE_IMPULSE_DRIVE_I', 'name': '...",[],[],"{'required': 0, 'capacity': 0}",GROWING
1,SHIP_LIGHT_SHUTTLE,Light Shuttle,The light shuttle is a versatile spacecraft de...,ABUNDANT,82671,"{'symbol': 'FRAME_SHUTTLE', 'name': 'Shuttle',...","{'symbol': 'REACTOR_CHEMICAL_I', 'name': 'Chem...","{'symbol': 'ENGINE_ION_DRIVE_I', 'name': 'Ion ...","[{'symbol': 'MODULE_CARGO_HOLD_II', 'name': 'E...","[{'symbol': 'MOUNT_TURRET_I', 'name': 'Rotary ...","{'required': 21, 'capacity': 40}",GROWING
2,SHIP_LIGHT_HAULER,Light Hauler,"A small, fast cargo ship that is designed for ...",ABUNDANT,279321,"{'symbol': 'FRAME_LIGHT_FREIGHTER', 'name': 'L...","{'symbol': 'REACTOR_CHEMICAL_I', 'name': 'Chem...","{'symbol': 'ENGINE_ION_DRIVE_I', 'name': 'Ion ...","[{'symbol': 'MODULE_CARGO_HOLD_II', 'name': 'E...","[{'symbol': 'MOUNT_TURRET_I', 'name': 'Rotary ...","{'required': 55, 'capacity': 80}",GROWING


In [55]:
sy_df[['type', 'modules']].explode('modules')

,type,modules
0,SHIP_PROBE,NaN
1,SHIP_LIGHT_SHUTTLE,"{'symbol': 'MODULE_CARGO_HOLD_II', 'name': 'Ex..."
1,SHIP_LIGHT_SHUTTLE,"{'symbol': 'MODULE_CREW_QUARTERS_I', 'name': '..."
2,SHIP_LIGHT_HAULER,"{'symbol': 'MODULE_CARGO_HOLD_II', 'name': 'Ex..."
2,SHIP_LIGHT_HAULER,"{'symbol': 'MODULE_CARGO_HOLD_II', 'name': 'Ex..."
2,SHIP_LIGHT_HAULER,"{'symbol': 'MODULE_CREW_QUARTERS_I', 'name': '..."
2,SHIP_LIGHT_HAULER,"{'symbol': 'MODULE_CREW_QUARTERS_I', 'name': '..."


In [65]:
sy_data

{'symbol': 'X1-TP30-A2',
 'shipTypes': [{'type': 'SHIP_PROBE'},
  {'type': 'SHIP_LIGHT_SHUTTLE'},
  {'type': 'SHIP_LIGHT_HAULER'}],
 'modificationsFee': 100,
 'transactions': [{'waypointSymbol': 'X1-TP30-A2',
   'shipSymbol': 'SHIP_PROBE',
   'shipType': 'SHIP_PROBE',
   'price': 45792,
   'agentSymbol': 'CHURT-LYNE',
   'timestamp': '2025-04-11T02:31:43.289Z'},
  {'waypointSymbol': 'X1-TP30-A2',
   'shipSymbol': 'SHIP_PROBE',
   'shipType': 'SHIP_PROBE',
   'price': 42964,
   'agentSymbol': 'CHURT-LYNE',
   'timestamp': '2025-04-11T02:23:42.298Z'},
  {'waypointSymbol': 'X1-TP30-A2',
   'shipSymbol': 'SHIP_PROBE',
   'shipType': 'SHIP_PROBE',
   'price': 40416,
   'agentSymbol': 'CHURT-LYNE',
   'timestamp': '2025-04-11T02:16:35.281Z'},
  {'waypointSymbol': 'X1-TP30-A2',
   'shipSymbol': 'SHIP_PROBE',
   'shipType': 'SHIP_PROBE',
   'price': 38120,
   'agentSymbol': 'CHURT-LYNE',
   'timestamp': '2025-04-11T01:57:36.324Z'},
  {'waypointSymbol': 'X1-TP30-A2',
   'shipSymbol': 'SHIP_PROB

In [71]:
# Parse shipyard data

ship_data = list()
module_data = list()

# For each ship
for s in sy_data['ships']:
    # Extract ship features
    ship_data.append({
        'shipyardSymbol': sy_data['symbol'],
        'type': s['type'],
        'name': s['name'],
        'description': s['description'],
        'purchasePrice': s['purchasePrice'],
        'supply': s['supply'],
        'frameSymbol': s['frame']['symbol'],
        'reactorSymbol': s['reactor']['symbol'],
        'engineSymbol': s['engine']['symbol'],
        'crewRequired': s['crew']['required'],
        'crewCapacity': s['crew']['capacity'],
        'activity': s['activity']
    })

    # Extract module features
    for m in s['modules']:
        module_data.append({
            'shipType': s['type'],
            'symbol': m['symbol'],
            'name': m['name'],
            'description': m['description'],
            'reqPower': m['requirements']['power'],
            'reqCrew': m['requirements']['crew'],
            'reqSlots': m['requirements']['slots'],
            'capacity': m['capacity']
        })

In [73]:
pd.DataFrame(module_data)

,shipType,symbol,name,description,reqPower,reqCrew,reqSlots,capacity
0,SHIP_LIGHT_SHUTTLE,MODULE_CARGO_HOLD_II,Expanded Cargo Hold,An expanded cargo hold module that provides mo...,2,2,2,40
1,SHIP_LIGHT_SHUTTLE,MODULE_CREW_QUARTERS_I,Crew Quarters,A module that provides living space and amenit...,1,2,1,40
2,SHIP_LIGHT_HAULER,MODULE_CARGO_HOLD_II,Expanded Cargo Hold,An expanded cargo hold module that provides mo...,2,2,2,40
3,SHIP_LIGHT_HAULER,MODULE_CARGO_HOLD_II,Expanded Cargo Hold,An expanded cargo hold module that provides mo...,2,2,2,40
4,SHIP_LIGHT_HAULER,MODULE_CREW_QUARTERS_I,Crew Quarters,A module that provides living space and amenit...,1,2,1,40
5,SHIP_LIGHT_HAULER,MODULE_CREW_QUARTERS_I,Crew Quarters,A module that provides living space and amenit...,1,2,1,40


In [85]:
print_nav_info('RYVIOS-2')

RYVIOS-2 is DOCKED at X1-TP30-C39.


In [88]:
F_trade.refresh_shipyard('RYVIOS-2')

True

In [97]:
F_nav.get_ship_nav('RYVIOS-1')

{'systemSymbol': 'X1-TP30',
 'waypointSymbol': 'X1-TP30-A3',
 'route': {'destination': {'symbol': 'X1-TP30-A3',
   'type': 'MOON',
   'systemSymbol': 'X1-TP30',
   'x': -19,
   'y': -14},
  'origin': {'symbol': 'X1-TP30-A1',
   'type': 'PLANET',
   'systemSymbol': 'X1-TP30',
   'x': -19,
   'y': -14},
  'departureTime': '2025-11-04T22:32:27.682Z',
  'arrival': '2025-11-04T22:32:42.682Z'},
 'status': 'IN_TRANSIT',
 'flightMode': 'BURN'}

In [102]:
datetime.fromisoformat('2025-11-04T22:32:42.682+00:00')

datetime.datetime(2025, 11, 4, 22, 32, 42, 682000, tzinfo=datetime.timezone.utc)

In [108]:
(datetime.fromisoformat('2025-11-04T22:32:42.682+00:00') - datetime.now(pytz.utc)).total_seconds() // 60

-5.0

In [ ]:
F_utils.ts_to_dt('2025-11-04T22:32:42.682Z') - datetime.now(pytz.utc)).total_seconds() // 60

datetime.datetime(2025, 11, 4, 22, 32, 42, 682000, tzinfo=<UTC>)

In [113]:
F_utils.ts_delta_seconds('2025-11-04T22:32:42.682Z') // 60

-12.0

In [161]:
F_nav.get_transit_time_seconds('RYVIOS-2')

2.909733

In [158]:
F_nav.get_ship_nav(ship)

{'systemSymbol': 'X1-TP30',
 'waypointSymbol': 'X1-TP30-C40',
 'route': {'destination': {'symbol': 'X1-TP30-C40',
   'type': 'FUEL_STATION',
   'systemSymbol': 'X1-TP30',
   'x': 76,
   'y': 87},
  'origin': {'symbol': 'X1-TP30-C39',
   'type': 'ORBITAL_STATION',
   'systemSymbol': 'X1-TP30',
   'x': 104,
   'y': 118},
  'departureTime': '2025-11-05T00:11:18.808Z',
  'arrival': '2025-11-05T00:13:30.808Z'},
 'status': 'IN_TRANSIT',
 'flightMode': 'CRUISE'}

In [162]:
F_nav.get_waypoints_in_system('X1-TP30')

[{'symbol': 'X1-TP30-A1',
  'type': 'PLANET',
  'systemSymbol': 'X1-TP30',
  'x': -19,
  'y': -14,
  'orbitals': [{'symbol': 'X1-TP30-A2'},
   {'symbol': 'X1-TP30-A3'},
   {'symbol': 'X1-TP30-A4'}],
  'traits': [{'symbol': 'TEMPERATE',
    'name': 'Temperate',
    'description': 'A world with a mild climate and balanced ecosystem, providing a comfortable environment for a variety of life forms and supporting diverse industries.'},
   {'symbol': 'SCATTERED_SETTLEMENTS',
    'name': 'Scattered Settlements',
    'description': 'A collection of dispersed communities, each independent yet connected through trade and communication networks.'},
   {'symbol': 'SALT_FLATS',
    'name': 'Salt Flats',
    'description': 'Expansive, barren plains covered in a thick layer of salt, offering unique opportunities for resource extraction, scientific research, and other activities.'},
   {'symbol': 'EXPLOSIVE_GASES',
    'name': 'Explosive Gases',
    'description': 'A volatile environment filled with h

In [164]:
constrsite = "X1-TP30-I55"
ST.get_request(f'/systems/X1-TP30/waypoints/{constrsite}/construction').json()

{'data': {'symbol': 'X1-TP30-I55',
  'materials': [{'tradeSymbol': 'FAB_MATS', 'required': 1600, 'fulfilled': 0},
   {'tradeSymbol': 'ADVANCED_CIRCUITRY', 'required': 400, 'fulfilled': 0},
   {'tradeSymbol': 'QUANTUM_STABILIZERS', 'required': 1, 'fulfilled': 1}],
  'isComplete': False}}

In [167]:
loops            = 0
interval_seconds = 60 * 10
selection_query  = \
                """
                select
                    *
                from TRADE_SYSTEM_MARGINS
                where 1=1
                    and margin > 1
                    and tradeVolume >= 10
                    and distance < 250
                    and src_supply in ("ABUNDANT", "HIGH")
                    and sink_supply in ("SCARCE", "LIMITED")
                    and symbol not in ("FAB_MATS", "ADVANCED_CIRCUITRY", "QUANTUM_STABILIZERS")
                    order by margin desc
                limit 1
                        """
    
# Try picking a route
candidates = pd.read_sql_query(selection_query, ST.DB_CONN())

In [ ]:
route_data = candidates.iloc[0].to_dict()
{route_data['symbol']:  max(route_data['tradeVolume'], 40)}

{'CLOTHING': 20}

In [173]:
route_data

{'symbol': 'CLOTHING',
 'purchasePrice': 2992,
 'sellPrice': 5147,
 'margin': 2155,
 'tradeVolume': 20,
 'source': 'X1-TP30-K87',
 'sink': 'X1-TP30-A1',
 'src_supply': 'HIGH',
 'sink_supply': 'SCARCE',
 'distance': 104.0,
 'ts_refreshed': '2025-11-04 23:15:25'}

In [4]:
import asyncio, time

In [ ]:
def make_async(f):
    """ Wrapper that hands blocking function f to a separate thread, effectively making it asynchronous. """
    async def asyncified(*args, **kwargs):
        return await asyncio.to_thread(f, *args, **kwargs)
    return asyncified

@make_async
def blocking1():
    print("Blocking 1 - 4s")
    time.sleep(4)

@make_async
def blocking2():
    print("Blocking 2 - 1s")
    time.sleep(1)

def blocking3():
    print("  Blocking 3")
    time.sleep(0.1)

def blocking4():
    print("  Blocking 4")
    time.sleep(0.1)

@make_async
def blocking5():
    print("Blocking 5 - 2s")
    blocking3()
    time.sleep(1.8)
    blocking4()

async def main():
    for _ in range(5):
        await asyncio.gather(blocking1(), blocking2())


In [24]:
await main()

Blocking 1 - 5s
Blocking 2 - 1s
Blocking 1 - 5s
Blocking 2 - 1s
Blocking 1 - 5sBlocking 2 - 1s

Blocking 1 - 5s
Blocking 2 - 1s
Blocking 1 - 5s
Blocking 2 - 1s


In [30]:
df = pd.read_json('./data/production-chain.json').explode('imports')
display(df)

,export,imports
0,ADVANCED_CIRCUITRY,ELECTRONICS
0,ADVANCED_CIRCUITRY,MICROPROCESSORS
1,AI_MAINFRAMES,ADVANCED_CIRCUITRY
1,AI_MAINFRAMES,MICROPROCESSORS
2,ALUMINUM,ALUMINUM_ORE
...,...,...
124,SUPERGRAINS,LAB_INSTRUMENTS
125,URANITE,URANITE_ORE
126,URANITE_ORE,EXPLOSIVES
127,VIRAL_AGENTS,POLYNUCLEOTIDES


In [31]:
df.to_sql("PRODUCTION_CHAIN", ST.DB_CONN())

253

In [33]:
F_nav._refresh_ship_nav('RYVIOS-1')

[ERROR] Failed to write nav data for RYVIOS-1. Exception:
no such table: ship.SHIP_NAV


OperationalError: no such table: ship.SHIP_NAV

In [11]:
data = {"lol": 2, "bar": "foo", "whatever": 0}
io.write_data(1234, data, mode='append', key=['lol'])

[ERROR] Exception while initialising table 1234:
near "1234": syntax error
[ERROR] Exception while initialising table 1234:
near "1234": syntax error
[ERROR] Exception while initialising table 1234:
near "1234": syntax error


False

In [5]:
data.values()

dict_values([2, 'foo', 0])

In [21]:
io._initiate_table("lol", data)

CREATE TABLE IF NOT EXISTS lol AS
SELECT
1 as lol,
"baz" as bar,
55 as whatever


True

In [5]:
io._table_exists("logs.REQUESTS")

True

In [10]:
F_nav._refresh_ship_nav('RYVIOS-3')

In [14]:
F_trade.refresh_tradegoods('RYVIOS-1')

True

In [10]:
F_nav.get_transit_time_seconds('RYVIOS-1')

QUERY:
 INSERT INTO 'logs.REQUESTS' VALUES (:url, :status_code, :request_body, :timestamp)

DATA:
 {'url': 'https://api.spacetraders.io/v2/my/ships/RYVIOS-1/nav', 'status_code': 200, 'request_body': '{}', 'timestamp': 1762455302.6274903}


0

In [30]:
n = F_nav.get_ship_nav('RYVIOS-1')
n

{'symbol': 'RYVIOS-1',
 'departureTime': '2025-11-06T22:02:19.912Z',
 'systemSymbol': 'X1-TP30',
 'waypointSymbol': 'X1-TP30-E43',
 'status': 'IN_TRANSIT',
 'arrival': '2025-11-06T22:04:00.912Z',
 'flightMode': 'CRUISE'}

In [ ]:
F_nav._refresh_ship_nav('RYVIOS-1')

In [31]:
F_nav.get_transit_time_seconds('RYVIOS-1')

-716.976149

In [34]:
(n['status'] == 'IN_TRANSIT') and (F_utils.ts_delta_seconds(n['arrival']))

-798.889057

In [49]:
F_nav.get_ship_nav('RYVIOS-1')

{'symbol': 'RYVIOS-1',
 'departureTime': '2025-11-06T23:25:45.642Z',
 'systemSymbol': 'X1-TP30',
 'waypointSymbol': 'X1-TP30-D41',
 'status': 'DOCKED',
 'arrival': '2025-11-06T23:26:40.641Z',
 'flightMode': 'CRUISE'}

In [48]:
F_nav._ship_nav_cache('RYVIOS-1')

{'symbol': 'RYVIOS-1',
 'departureTime': '2025-11-06T23:25:45.642Z',
 'systemSymbol': 'X1-TP30',
 'waypointSymbol': 'X1-TP30-D41',
 'status': 'DOCKED',
 'arrival': '2025-11-06T23:26:40.641Z',
 'flightMode': 'CRUISE'}

In [47]:
F_nav._refresh_ship_nav('RYVIOS-1')

True

In [41]:
from SpaceTraders import scripts

In [45]:
await scripts.await_navigation('RYVIOS-1')

In [46]:
F_nav._ship_nav_cache('RYVIOS-1')

{'symbol': 'RYVIOS-1',
 'departureTime': '2025-11-06T23:18:58.534Z',
 'systemSymbol': 'X1-TP30',
 'waypointSymbol': 'X1-TP30-H52',
 'status': 'IN_ORBIT',
 'arrival': '2025-11-06T23:19:53.534Z',
 'flightMode': 'CRUISE'}

In [3]:
F_nav.get_ship_nav('RYVIOS-3')

{'symbol': 'RYVIOS-3',
 'systemSymbol': 'X1-TP30',
 'waypointSymbol': 'X1-TP30-AB5E',
 'departureTime': '2025-11-04T00:40:11.786Z',
 'arrival': '2025-11-04T00:41:50.786Z',
 'flightMode': 'CRUISE',
 'status': 'IN_ORBIT'}

In [30]:
F_nav._refresh_ship_nav('RYVIOS-1')
F_nav.get_ship_nav('RYVIOS-1')

{'departureTime': '2025-11-06T23:47:30.436Z',
 'flightMode': 'CRUISE',
 'arrival': '2025-11-06T23:48:25.436Z',
 'waypointSymbol': 'X1-TP30-H52',
 'systemSymbol': 'X1-TP30',
 'symbol': 'RYVIOS-1',
 'status': 'DOCKED'}

In [7]:
F_nav.get_ship_nav('RYVIOS-2')

{'symbol': 'RYVIOS-2',
 'systemSymbol': 'X1-TP30',
 'waypointSymbol': 'X1-TP30-C39',
 'departureTime': '2025-11-06T23:39:54.928Z',
 'arrival': '2025-11-07T00:17:03.928Z',
 'flightMode': 'CRUISE',
 'status': 'IN_ORBIT'}

In [4]:
F_nav.dock_ship('RYVIOS-2')

[ERROR] Exception during table update.


True

In [ ]:
{'data': {'nav': {'systemSymbol': 'X1-TP30', 'waypointSymbol': 'X1-TP30-C39', 'route': {'destination': {'symbol': 'X1-TP30-C39', 'type': 'ORBITAL_STATION', 'systemSymbol': 'X1-TP30', 'x': 104, 'y': 118}, 'origin': {'symbol': 'X1-TP30-J58', 'type': 'ASTEROID_BASE', 'systemSymbol': 'X1-TP30', 'x': -578, 'y': 431}, 'departureTime': '2025-11-06T23:39:54.928Z', 'arrival': '2025-11-07T00:17:03.928Z'}, 'status': 'DOCKED', 'flightMode': 'CRUISE'}}}['data']['nav']['status']

In [24]:
data = [{'symbol': 'RYVIOS-1', 'status': 'IN_ORBIT'}]
print(io.update_records('ship.SHIP_NAV', data, ['symbol']))

True


In [37]:
F_nav.orbit_ship('RYVIOS-1')

True

In [15]:
F_nav.get_transit_time_seconds('RYVIOS-1')

72.237954

In [21]:
F_trade.get_total_profit_from_trade('RYVIOS-1', 'X1-TP30-D41', 'X1-TP30-H52', '2025-11-07T09:27:00Z')

49365

In [ ]:
F_utils.ts_to_dt()

In [22]:
from datetime import timezone

In [24]:
datetime.now(timezone.utc).strftime('%Y-%m-%dT%H:%M:%SZ')

'2025-11-07T10:11:20Z'

In [25]:
ST.get_request('/systems/X1-TP30/waypoints/X1-TP30-I55/construction').json()

{'data': {'symbol': 'X1-TP30-I55',
  'materials': [{'tradeSymbol': 'FAB_MATS', 'required': 1600, 'fulfilled': 0},
   {'tradeSymbol': 'ADVANCED_CIRCUITRY', 'required': 400, 'fulfilled': 0},
   {'tradeSymbol': 'QUANTUM_STABILIZERS', 'required': 1, 'fulfilled': 1}],
  'isComplete': False}}

In [36]:
selection_query  = \
                        """
                        select
                            *
                        from SUPPLY_CHAIN_TRADES
                        where 1=1
                            and target_good in ("FAB_MATS", "ADVANCED_CIRCUITRY")
                            and target_market like "X1-TP30-%"
                            and margin > 2
                            and imported_good_supply not in ("ABUNDANT", "HIGH")
                            and export_supply not in ("SCARCE", "LIMITED")
                        order by margin desc
                        limit 1
                        """

candidates = io.read_dict(selection_query)

In [19]:
ST.get_ship_info('RYVIOS-3')['fuel']['capacity']

80

In [25]:
F_nav.get_path('RYVIOS-2', 'X1-TP30-H51', 'X1-TP30-A1')

('X1-TP30-A1', 'DRIFT', 59.21148537234985)

In [24]:
ST.get_ship_info('RYVIOS-2')

{'symbol': 'RYVIOS-2',
 'registration': {'name': 'RYVIOS-2',
  'factionSymbol': 'COSMIC',
  'role': 'SATELLITE'},
 'nav': {'systemSymbol': 'X1-TP30',
  'waypointSymbol': 'X1-TP30-J57',
  'route': {'destination': {'symbol': 'X1-TP30-J57',
    'type': 'FUEL_STATION',
    'systemSymbol': 'X1-TP30',
    'x': -479,
    'y': 358},
   'origin': {'symbol': 'X1-TP30-I55',
    'type': 'JUMP_GATE',
    'systemSymbol': 'X1-TP30',
    'x': -362,
    'y': 270},
   'departureTime': '2025-11-07T14:57:20.857Z',
   'arrival': '2025-11-07T15:04:52.857Z'},
  'status': 'IN_TRANSIT',
  'flightMode': 'CRUISE'},
 'crew': {'current': 0,
  'required': 0,
  'capacity': 0,
  'rotation': 'STRICT',
  'morale': 100,
  'wages': 0},
 'frame': {'symbol': 'FRAME_PROBE',
  'name': 'Probe',
  'condition': 0.979263151820819,
  'integrity': 1,
  'description': 'A small, unmanned spacecraft used for exploration, reconnaissance, and scientific research.',
  'moduleSlots': 0,
  'mountingPoints': 0,
  'fuelCapacity': 0,
  'requ

In [26]:
r = ST.post_request('/my/ships', {'shipType': "SHIP_LIGHT_HAULER",  'waypointSymbol': "X1-TP30-A2"})

In [27]:
r.json()

{'data': {'ship': {'symbol': 'RYVIOS-4',
   'registration': {'name': 'RYVIOS-4',
    'factionSymbol': 'COSMIC',
    'role': 'HAULER'},
   'nav': {'systemSymbol': 'X1-TP30',
    'waypointSymbol': 'X1-TP30-A2',
    'route': {'destination': {'symbol': 'X1-TP30-A2',
      'type': 'MOON',
      'systemSymbol': 'X1-TP30',
      'x': -19,
      'y': -14},
     'origin': {'symbol': 'X1-TP30-A2',
      'type': 'MOON',
      'systemSymbol': 'X1-TP30',
      'x': -19,
      'y': -14},
     'departureTime': '2025-11-07T15:33:33.275Z',
     'arrival': '2025-11-07T15:33:33.275Z'},
    'status': 'DOCKED',
    'flightMode': 'CRUISE'},
   'crew': {'current': 55,
    'required': 55,
    'capacity': 80,
    'rotation': 'STRICT',
    'morale': 100,
    'wages': 0},
   'frame': {'symbol': 'FRAME_LIGHT_FREIGHTER',
    'name': 'Light Freighter',
    'condition': 1,
    'integrity': 1,
    'description': 'A small, versatile spacecraft used for cargo transport and other commercial operations.',
    'moduleSlot

In [28]:
F_nav.get_fuel_capacity('RYVIOS-4')

600

In [30]:
F_trade.get_ship_cargo('RYVIOS-1')

{'capacity': 40,
 'units': 40,
 'inventory': [{'symbol': 'COPPER',
   'name': 'Copper',
   'description': 'Copper is an essential component in the production of electronic components and wiring.',
   'units': 40}]}

In [6]:
wp = F_nav.get_waypoints_in_system('X1-QT34')
print(len(wp))

87


In [7]:
df = io.parse_nested_obj(wp[0], "waypoint")

In [8]:
display(df['traits'])

,symbol,name,description
0,VOLCANIC,Volcanic,A volatile world marked by intense volcanic ac...
1,OUTPOST,Outpost,"A small, remote settlement providing essential..."
2,VIBRANT_AURORAS,Vibrant Auroras,A celestial light show caused by the interacti...
3,CANYONS,Canyons,"Deep, winding ravines carved into the landscap..."
4,DRY_SEABEDS,Dry Seabeds,"Vast, desolate landscapes that once held ocean..."
5,MARKETPLACE,Marketplace,A thriving center of commerce where traders fr...


In [32]:
df['waypoint']['symbol'].to_json(orient="records")

'["X1-QT34-A1"]'

In [37]:
io.write_df("nav.WAYPOINTS", df['waypoint'], mode="update", key=["symbol"])

[{'symbol': 'X1-QT34-A1'}]
DELETE FROM 'nav.WAYPOINTS'
WHERE
symbol = :symbol


True

In [22]:
df['waypoint']#.to_sql('nav.WAYPOINTS', io._DB_CONN(), if_exists="append", index=False)

,symbol,type,systemSymbol,x,y,isUnderConstruction,faction,chart
0,X1-QT34-A1,PLANET,X1-QT34,23,-13,False,{'symbol': 'COSMIC'},"{'waypointSymbol': 'X1-QT34-A1', 'submittedBy'..."


In [36]:
df["waypoint"][["symbol", "systemSymbol"]].drop_duplicates().to_dict(orient="records")

[{'symbol': 'X1-QT34-A1', 'systemSymbol': 'X1-QT34'}]

In [5]:
F_nav._refresh_waypoints('X1-QT34')

ValueError: No objects to concatenate

In [6]:
# Get the paginated waypoint data
system = "X1-QT34"
nextpage = 1
data = list()
total = -1
while total < 0 or len(data) < total:
    # Fetch next page
    r = ST.get_request(f'/systems/{system}/waypoints', params={'page': nextpage})
    if r.status_code == 200:
        data.extend(r.json()['data'])
        nextpage += 1
        total = r.json()['meta']['total']
    else:
        print(f'[WARNING] Failed to fetch page {nextpage} of waypoints in system {system}.')
        print(f'   [INFO]', r.json())    

In [24]:
F_nav._refresh_waypoints('X1-QT34')

In [34]:
F_nav.get_path('RYVIOS-2', "X1-QT34-H57", "X1-QT34-H56")

[('X1-QT34-H56', 'DRIFT', 0.0)]

In [31]:
F_nav.wp_distance("X1-QT34-H57", "X1-QT34-H56")

0.0

In [38]:
F_trade.refresh_shipyard('RYVIOS-2')

[ERROR] Unable to fetch market info for X1-QT34-AX5Z.
 [INFO] {'error': {'message': 'Shipyard at waypoint X1-QT34-AX5Z not found.', 'code': 404}}
[ERROR] RYVIOS-2 failed to refresh shipyard data for X1-QT34-AX5Z.


False

In [39]:
l = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [43]:
l[6:] + l[:6]

[7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6]

In [102]:
# Get all satellites in system
system = "X1-QT34"

q = """
    select
        nav.symbol
    from 'ship.REGISTRATION' reg
    inner join 'ship.NAV' nav
        on reg.role = "SATELLITE" and reg.shipSymbol = nav.symbol
    """

probes = [r[0] for r in io.read_list(q)]

In [103]:
probes

['RYVIOS-2']

In [ ]:
lock table:
- shipSymbol
- controller
- priority
- blocked 

In [112]:
io.update_records("control.SHIP_LOCKS", {"shipSymbol": "RYVIOS-1", "blocked": True}, key_cols=["shipSymbol"])

True

In [167]:
from SpaceTraders import fleet_resource_manager

In [151]:
fleet_resource_manager.release_ship("RYVIOS-1")

True

In [129]:
fleet_resource_manager.set_ship_blocked_status("RYVIOS-1", True)

True

In [153]:
fleet_resource_manager.lock_ship("RYVIOS-1", "MANUAL", 1)

True

In [132]:
fleet_resource_manager.get_ship_blocked_status("RYVIOS-1")

False

In [137]:
fleet_resource_manager.get_ship_controller("RYVIOS-1")

(None, -1)

In [269]:
# Check all available probes
BASE_PRIO_MGR_PROBES = 100
probes = fleet_resource_manager.get_available_ships_in_systems([system], ship_role="SATELLITE", prio=BASE_PRIO_MGR_PROBES)

In [270]:
probes

[]

In [200]:
# Check which markets need updating most urgently
q = """
    with market_update_times as (
    select
        marketSymbol,
        min(ts_created) as ts_last_update,
        datetime(min(ts_created), 'unixepoch', 'localtime') as last_update
    from tradegoods_current
    group by marketSymbol
    )

    select
        distinct wp.symbol
    from 'nav.WAYPOINTS' wp

    inner join 'nav.TRAITS' t
    on wp.symbol = t.waypointSymbol
    and t.symbol = "MARKETPLACE"

    left join market_update_times mu
    on mu.marketSymbol = wp.symbol

    order by mu.ts_last_update asc
    """
market_order = [r[0] for r in io.read_list(q)]

In [201]:
# Request every available ship
acquired = list()
for p in probes:
    if fleet_resource_manager.request_ship(p, "PROBE-MANAGER-X1-QT34", BASE_PRIO_MGR_PROBES):
        acquired.append(p)

In [210]:
acquired = ["RYVIOS-1", "A", "B", "C", "D"]

In [213]:
# Dummy update function
import random, asyncio
async def market_refresh(ship, market):
    print(f"{ship} is refreshing {market}.")
    delay = random.randint(20, 200) / 100.0
    await asyncio.sleep(1.5 + delay)
    print(f"{ship} has refreshed {market}.")
    return (random.random() >= 0.1)

In [214]:
# Assign each ship to a market (preferably by current distance)
order_assignment = [(acquired[i], market_order[i]) for i in range(len(acquired))]

In [217]:
# Refresh the top N most outdated markets
tasks = [asyncio.create_task(market_refresh(*assigment)) for assigment in order_assignment]
results = await asyncio.gather(*tasks)

RYVIOS-1 is refreshing X1-QT34-J63.
A is refreshing X1-QT34-J62.
B is refreshing X1-QT34-I60.
C is refreshing X1-QT34-I61.
D is refreshing X1-QT34-E46.
RYVIOS-1 has refreshed X1-QT34-J63.
B has refreshed X1-QT34-I60.
A has refreshed X1-QT34-J62.
D has refreshed X1-QT34-E46.
C has refreshed X1-QT34-I61.


In [223]:
sum([True, True, False, True, False])

3

In [224]:
# Release probes
for p in acquired:
    fleet_resource_manager.release_ship(p)

In [240]:
fleet_resource_manager.release_ship('RYVIOS-3')

True

In [273]:
for p in ['RYVIOS-1', 'RYVIOS-2', 'RYVIOS-3']:
    print(p, ':', F_nav.get_transit_time_seconds(p))

RYVIOS-1 : 0
RYVIOS-2 : 445.891197
RYVIOS-3 : 0


In [243]:
fleet_resource_manager.request_ship('RYVIOS-2', 'USER', 0)

[WARNING] Fleet resources has detected a moving ship without controller: RYVIOS-2.


False